In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import re

def scrap_other_info(r,company,other_info_table):

  try:
    row = other_info_table.find_all('tr')[r]
    cells = row.find_all('td')
    date = cells[0].text.strip()
    pattern = r'\b\w{3} \d{2}, \d{4}\b'
    # Search for the date pattern in the text
    match = re.search(pattern, date)
    if match:
      date = match.group()
      data = cells[1].text.strip()

      #regex construction
      sponsor_director_pattern = r'Sponsor/Director:\s+([\d.]+)'
      govt_pattern = r'Govt:\s+([\d.]+)'
      institute_pattern = r'Institute:\s+([\d.]+)'
      foreign_pattern = r'Foreign:\s+([\d.]+)'
      public_pattern = r'Public:\s+([\d.]+)'

      # Find the matches using regex
      sponsor_director_match = re.findall(sponsor_director_pattern, data)
      govt_match = re.findall(govt_pattern, data)
      institute_match = re.findall(institute_pattern, data)
      foreign_match = re.findall(foreign_pattern, data)
      public_match = re.findall(public_pattern, data)

      # Extracing the values from the matches and convert them to float
      sponsor_director_values = [float(value.strip()) for value in sponsor_director_match]
      govt_values = [float(value.strip()) for value in govt_match]
      institute_values = [float(value.strip()) for value in institute_match]
      foreign_values = [float(value.strip()) for value in foreign_match]
      public_values = [float(value.strip()) for value in public_match]
      List = [company,date,*sponsor_director_values,*govt_values,*institute_values,*foreign_values,*public_values]

      return List

  except Exception as e:
    # Handle any other exceptions
    print("An error occurred:", str(e))
url = "https://www.dsebd.org/company_listing.php"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
all_body_content = soup.find_all("div", {"class": "BodyContent"})
company_list = []
for com in all_body_content:
  code = com.find_all("a")
  for only_code in code:

    company_list.append(only_code.text.strip())
company_list = [x for x in company_list if not x.startswith("TB")]
company_list = [x for x in company_list if not x.startswith("More...")]
print(company_list)
company_list_df = pd.DataFrame(company_list)
company_list_df = company_list_df.rename(columns={0: 'Company Code'})
print(company_list_df)
company_details = []
other_info = []
for company in company_list:
  row_data = []
  baseurl = "https://www.dsebd.org/displayCompany.php?name="+company

  page = requests.get(baseurl)

  soup = BeautifulSoup(page.content, "html.parser")

  #finding company names under h2 header of class BodyHead

  companyname_header = soup.find('h2', {'class': 'BodyHead'})


  com_name = companyname_header.text.strip()[companyname_header.text.strip().find(": ") + len(": "):]
  row_data.append(com_name)


  com_table = soup.find('table', {'class': 'shares-table'})
  for row in com_table.find_all('tr'):
    for cell in row.find_all(['th']):
        index = cell.text.strip().find(": ")
        substring = cell.text.strip()[index + len(": "):]
        row_data.append(substring)
  tables = soup.find_all('div', {'class': 'table-responsive'})
  sector_table_info = tables[2]
  r = sector_table_info.find_all('tr')[3]
  cells = r.find_all('td')
  sector = cells[1].text.strip()
  row_data.append(sector)
  company_details.append(row_data)
 #extracting other info

  other_info_table = tables[9]
  for num in [3, 5, 7]:
      row = []
      l = scrap_other_info(num,company,other_info_table)
      other_info.append(l) #adding the data

other_info = [value for value in filter(None, other_info)]
company_info_df = pd.DataFrame(other_info,columns =['Trading Code', 'Date','Sponsor/Director','Govt','Institute','Foreign','Public']) #creating df

# print(company_info_df)
#df to csv
file_path = '/content/holdings.csv'
company_info_df.to_csv(file_path, index=False)
company_details_df = pd.DataFrame(company_details, columns =['Company Name', 'Trading Code','Scrip Code','Sector'])

print(company_details_df)
file_path = '/content/companies.csv'
company_details_df.to_csv(file_path, index=False)

['AAMRANET', 'AAMRATECH', 'ABB1STMF', 'ABBANK', 'ABBLPBOND', 'ACFL', 'ACI', 'ACIFORMULA', 'ACMELAB', 'ACMEPL', 'ACTIVEFINE', 'ADNTEL', 'ADVENT', 'AFCAGRO', 'AFTABAUTO', 'AGNISYSL', 'AGRANINS', 'AIBL1STIMF', 'AIBLPBOND', 'AIL', 'AL-HAJTEX', 'ALARABANK', 'ALIF', 'ALLTEX', 'AMANFEED', 'AMBEEPHA', 'AMCL(PRAN)', 'ANLIMAYARN', 'ANWARGALV', 'AOL', 'APEXFOODS', 'APEXFOOT', 'APEXSPINN', 'APEXTANRY', 'APOLOISPAT', 'APSCLBOND', 'ARAMIT', 'ARAMITCEM', 'ARGONDENIM', 'ASIAINS', 'ASIAPACINS', 'ATCSLGF', 'ATLASBANG', 'AZIZPIPES', 'BANGAS', 'BANKASIA', 'BARKAPOWER', 'BATASHOE', 'BATBC', 'BAYLEASING', 'BBS', 'BBSCABLES', 'BDAUTOCA', 'BDCOM', 'BDFINANCE', 'BDLAMPS', 'BDSERVICE', 'BDTHAI', 'BDTHAIFOOD', 'BDWELDING', 'BEACHHATCH', 'BEACONPHAR', 'BENGALWTL', 'BERGERPBL', 'BEXGSUKUK', 'BEXIMCO', 'BGIC', 'BIFC', 'BNICL', 'BPML', 'BPPL', 'BRACBANK', 'BSC', 'BSCCL', 'BSRMLTD', 'BSRMSTEEL', 'BXPHARMA', 'BXSYNTH', 'CAPMBDBLMF', 'CAPMIBBLMF', 'CBLPBOND', 'CENTRALINS', 'CENTRALPHL', 'CITYBANK', 'CITYGENINS', 'CLICL

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
headers = ["company_ID", "company_name", "scrip_code", "trading_code", "sector", "url", "address"]
df = pd.DataFrame(columns=headers)

url = "https://www.dsebd.org/company_listing.php"

response = requests.get(url) # Send a GET request to the website


soup = BeautifulSoup(response.content, "html.parser") #beautifulsoup object created

# Find the HTML elements that contain the company code
results = soup.find_all("div", {"class" : "BodyContent"})
trading_code = soup.find_all("a", {"class" : "ab1"})
t_code= []
for result in results: #iterate and get trading codes
    code = result.find_all("a")
    for getcode in code:
        t_code.append(getcode.text.strip()) #appending the value that we got into the emply list
        print(getcode.text.strip())

In [ ]:
address_list=[]
for result in results:
  code = result.find_all("a")
  for name in code:
    page1 = requests.get("https://dsebd.org/displayCompany.php?name=" + name.text.strip())
    soup = BeautifulSoup(page1.content, 'html.parser')
    sector = soup.find('td', string='Web Address')
    category = sector.find_next_siblings('td')
    for td in category:
              address = td.text.strip()
              if address:
                  address_list.append(address)
              else:
                  address_list.append(None)
              print(address)

In [ ]:
url_list=[]
for result in results:
  code = result.find_all("a")
  for name in code:
    p = ('https://dsebd.org/displayCompany.php?name=' + name.text.strip())
    url_list.append(p)
# print(url_list)


In [ ]:


# Step 1: Load the CSV file into a DataFrame
df = pd.read_csv('companies.csv')

# Step 2: Modify the DataFrame by inserting the desired column
new_column = ['url', 'address']  # Example column values
df['url'] = url_list
df['address'] = address_list
df['new_column_name'] = new_column

# Step 3: Save the modified DataFrame back to the CSV file
df.to_csv('companies.csv', index=False)
